In [1]:
import requests
import re
import urllib.request
from urllib.parse import urlparse
import duckdb
import os
import xml.etree.ElementTree as ET
from datetime import date, timedelta

import boto3
import requests

In [2]:
def get_rss():
    print("Downloading RSS...")
    req = urllib.request.Request("https://movilidad-opendata.mitma.es/RSS.xml", headers={"User-Agent": "MITMA-RSS-parser"})
    txt = urllib.request.urlopen(req).read().decode("utf-8", "ignore")
    os.makedirs("data", exist_ok=True)
    with open("data/RSS.xml", "w") as f:
        f.write(txt)
    return ET.XML(txt)

def fetch_trips_urls(datestring, root=None):
    table_title = "Viajes_distritos"
    urls = []
    if root is None:
        root = get_rss()
    for item in root.findall("./channel/item"):
        title = item.find('title').text
        url = item.find("link").text
        if "estudios_basicos" in url and re.search(f"{table_title}", title) and re.search(datestring, title):
            urls.append(url)
    return urls


def read_url(url, con):
    #source_query = f"SELECT * FROM read_csv('{url}', all_varchar=True, ignore_errors=True)"
    con.sql(f"""
        CREATE TABLE IF NOT EXISTS prueba AS 
        (SELECT * FROM read_csv('{url}', all_varchar=True))
    """)


In [9]:
def get_misc_urls():
    urls = []
    root = get_rss()
    for item in root.findall("./channel/item"):
        title = item.find('title').text
        url = item.find("link").text
        if "nombres_distritos.csv" in title:
            urls.append(url)
        if "nombres_gaus.csv" in title:
            urls.append(url)
        if "nombres_municipios.csv" in title:
            urls.append(url)
        if "relacion_ine_zonificacionMitma.csv" in title:
            urls.append(url)

    urls.append('s3://carlos-s3-bdet-ducklake/festivos/festivos.csv')
    return urls

In [25]:
def fetch_trips_url(day, root):
    datestring = day.replace("-", "")
    table_title = "Viajes_distritos"
    urls = []
    for item in root.findall("./channel/item"):
        title = item.find('title').text
        url = item.find("link").text
        if "estudios_basicos" in url and re.search(f"{table_title}", title) and re.search(datestring, title):
            return (url, day)

def get_trips_range():
    date_start = date(2023,1,1)
    date_end = date(2023, 12, 31)
    root = get_rss()
    print("root done")
    urls = []
    delta = date_end - date_start   # returns timedelta

    for i in range(delta.days + 1):
        day = date_start + timedelta(days=i)
        url = fetch_trips_url(str(day), root)
        if url:
            urls.append(url)

    return urls

In [32]:
con = duckdb.connect()

query = f"""
                            CREATE OR REPLACE TABLE {'bronze_relacion_mitma'} AS
                            SELECT 
                                seccion_ine,
                                distrito_ine,
                                municipio_ine,
                                distrito_mitma,
                                municipio_mitma,
                                gau_mitma
                            FROM read_csv('{"https://movilidad-opendata.mitma.es/zonificacion/relacion_ine_zonificacionMitma.csv"}', 
                                        auto_detect=TRUE, 
                                        header=TRUE,
                                        sep='|')
                            WHERE distrito_ine IS DISTINCT FROM 'NA'
                                        """

con.sql(query)

con.sql("SELECT * from bronze_relacion_mitma LIMIT 10")

┌─────────────┬──────────────┬───────────────┬────────────────┬─────────────────┬───────────┐
│ seccion_ine │ distrito_ine │ municipio_ine │ distrito_mitma │ municipio_mitma │ gau_mitma │
│   varchar   │   varchar    │    varchar    │    varchar     │     varchar     │  varchar  │
├─────────────┼──────────────┼───────────────┼────────────────┼─────────────────┼───────────┤
│ 0100101001  │ 0100101      │ 01001         │ 01001          │ 01001           │ 01001     │
│ 0100101002  │ 0100101      │ 01001         │ 01001          │ 01001           │ 01001     │
│ 0100201001  │ 0100201      │ 01002         │ 01002          │ 01002           │ 01002     │
│ 0100201002  │ 0100201      │ 01002         │ 01002          │ 01002           │ 01002     │
│ 0100201003  │ 0100201      │ 01002         │ 01002          │ 01002           │ 01002     │
│ 0100201004  │ 0100201      │ 01002         │ 01002          │ 01002           │ 01002     │
│ 0100201005  │ 0100201      │ 01002         │ 01002        

In [48]:
query = """CREATE OR REPLACE TABLE bronze_festivos AS
                            SELECT 
                                fecha,
                                es_festivo,
                                es_fin_de_semana
                            FROM read_csv('data/festivos.csv', 
                                        auto_detect=TRUE, 
                                        header=TRUE);"""

con.sql(query)

In [49]:
con.sql("SELECT * FROM bronze_festivos LIMIT 10")

┌────────────┬────────────┬──────────────────┐
│   fecha    │ es_festivo │ es_fin_de_semana │
│    date    │   int64    │      int64       │
├────────────┼────────────┼──────────────────┤
│ 2023-01-01 │          0 │                1 │
│ 2023-01-02 │          0 │                0 │
│ 2023-01-03 │          0 │                0 │
│ 2023-01-04 │          0 │                0 │
│ 2023-01-05 │          0 │                0 │
│ 2023-01-06 │          1 │                0 │
│ 2023-01-07 │          0 │                1 │
│ 2023-01-08 │          0 │                1 │
│ 2023-01-09 │          0 │                0 │
│ 2023-01-10 │          0 │                0 │
├────────────┴────────────┴──────────────────┤
│ 10 rows                          3 columns │
└────────────────────────────────────────────┘

In [50]:
query = """--sql
            CREATE OR REPLACE TABLE silver_festivos AS
                SELECT 
                    fecha as date,
                    CAST(es_festivo AS BOOLEAN) as is_holiday,
                    CAST(es_fin_de_semana AS BOOLEAN) as is_weekend
                FROM bronze_festivos"""

con.sql(query)

In [51]:
con.sql("SELECT * FROM silver_festivos LIMIT 10")

┌────────────┬────────────┬────────────┐
│    date    │ is_holiday │ is_weekend │
│    date    │  boolean   │  boolean   │
├────────────┼────────────┼────────────┤
│ 2023-01-01 │ false      │ true       │
│ 2023-01-02 │ false      │ false      │
│ 2023-01-03 │ false      │ false      │
│ 2023-01-04 │ false      │ false      │
│ 2023-01-05 │ false      │ false      │
│ 2023-01-06 │ true       │ false      │
│ 2023-01-07 │ false      │ true       │
│ 2023-01-08 │ false      │ true       │
│ 2023-01-09 │ false      │ false      │
│ 2023-01-10 │ false      │ false      │
├────────────┴────────────┴────────────┤
│ 10 rows                    3 columns │
└──────────────────────────────────────┘

In [39]:
con.sql("SELECT * FROM silver_relacion_ine LIMIT 10")

┌─────────────┬──────────────┬───────────────┬────────────────┬─────────────────┬───────────┐
│ seccion_ine │ distrito_ine │ municipio_ine │ distrito_mitma │ municipio_mitma │ gau_mitma │
│   varchar   │   varchar    │    varchar    │    varchar     │     varchar     │  varchar  │
├─────────────┼──────────────┼───────────────┼────────────────┼─────────────────┼───────────┤
│ 0100101001  │ 0100101      │ 01001         │ 01001          │ 01001           │ 01001     │
│ 0100101002  │ 0100101      │ 01001         │ 01001          │ 01001           │ 01001     │
│ 0100201001  │ 0100201      │ 01002         │ 01002          │ 01002           │ 01002     │
│ 0100201002  │ 0100201      │ 01002         │ 01002          │ 01002           │ 01002     │
│ 0100201003  │ 0100201      │ 01002         │ 01002          │ 01002           │ 01002     │
│ 0100201004  │ 0100201      │ 01002         │ 01002          │ 01002           │ 01002     │
│ 0100201005  │ 0100201      │ 01002         │ 01002        

In [ ]:
def fetch_trips_urls_(datestring, root=None):
    table_title = "Viajes_distritos"
    urls = []
    if root is None:
        root = get_rss()
    for item in root.findall("./channel/item"):
        title = item.find('title').text
        url = item.find("link").text
        if "estudios_basicos" in url and re.search(f"{table_title}", title) and re.search(datestring, title):
            urls.append(url)
    return urls

def fetch_trips_url(day, root):
    datestring = day.replace("-", "")
    table_title = "Viajes_distritos"
    urls = []
    for item in root.findall("./channel/item"):
        title = item.find('title').text
        url = item.find("link").text
        if "estudios_basicos" in url and re.search(f"{table_title}", title) and re.search(datestring, title):
            return (url, day)


def get_trips_range(date_start, date_end):
    root = get_rss()
    print("root done")
    urls = []
    delta = date_end - date_start   # returns timedelta

    for i in range(delta.days + 1):
        day = date_start + timedelta(days=i)
        urls.append(fetch_trips_url(str(day), root))

    return urls


In [41]:
a= (2023, 1, 1)

date(*a)

datetime.date(2023, 1, 1)

In [17]:
urls = fetch_trips_urls(r"2023[0-1][0-9][0-3][0-9]")

In [67]:
nombre_archivo = os.path.basename(f"https://servicios.ine.es/wstempus/js/es/DATOS_TABLA/30656.json")

In [101]:
con = duckdb.connect()
con.sql("INSTALL json; LOAD json;")

query_ine = """--sql

CREATE OR REPLACE TABLE bronze_ine_renta AS

SELECT 
    -- Python: codigo = meta[0].get("Codigo")
    -- SQL: Los arrays empiezan en 1. Accedemos al primer elemento de la lista MetaData.
    MetaData[1].Codigo as codigo,
    
    -- Python: dato.get("Fecha") (Esto viene del UNNEST de abajo)
    d.Fecha as fecha,
    
    -- Python: dato.get("Valor")
    -- Convertimos a número por si acaso viene como texto
    TRY_CAST(d.Valor AS DOUBLE) as valor

FROM read_json_auto('data/ine/30656.json')

-- 4. LA MAGIA: El "flattening" (aplanado)
-- Esto equivale a tu bucle "for dato in item.get('Data')"
-- Crea una fila nueva por cada elemento dentro de la lista 'Data'
CROSS JOIN UNNEST(Data) AS t(d)

-- 5. EL FILTRO: Tu lógica de IF
WHERE 
    -- Python: meta[0].get("T3_Variable") == "Secciones"
    MetaData[1].T3_Variable = 'Secciones'
    
    -- Python: meta[2].get("Nombre") == "Renta..." (Recordamos: índice 2 -> 3 en SQL)
    AND MetaData[3].Nombre = 'Renta neta media por persona';
"""

query_ine_2 = """--sql
-- 2. Creación de la Tabla
CREATE OR REPLACE TABLE bronze_ine_poblacion AS

SELECT 
    -- Python: meta[0].get("Codigo") -> SQL: Index 1
    MetaData[1].Codigo as codigo,
    
    -- Datos del UNNEST
    d.Fecha as fecha,
    TRY_CAST(d.Valor AS DOUBLE) as valor

FROM read_json_auto('data/ine/66595.json') -- Ojo al nombre del archivo
CROSS JOIN UNNEST(Data) AS t(d)

WHERE 
    -- Traducción directa de tus IFs de Python (Indice Python + 1 = Indice SQL)
    
    -- Python: meta[0].get("T3_Variable") == "Secciones"
    MetaData[1].T3_Variable = 'Secciones'
    
    -- Python: meta[1].get("Nombre") == "16 y más años"
    AND MetaData[2].Nombre = '16 y más años'
    
    -- Python: meta[2].get("Nombre") == "Total"
    AND MetaData[3].Nombre = 'Total'
    
    -- Python: meta[3].get("Nombre") == "Total"
    AND MetaData[4].Nombre = 'Total'
    
    -- Python: meta[4].get("Nombre") == "Total"
    AND MetaData[5].Nombre = 'Total';
"""

In [102]:
con.sql(query_ine)

In [107]:
con.sql("SELECT MetaData[1].T3_Variable FROM read_json('data/ine/30656.json', auto_detect=True) limit 10")

┌───────────────────────────┐
│ (MetaData[1]).T3_Variable │
│          varchar          │
├───────────────────────────┤
│ Municipios                │
│ Municipios                │
│ Municipios                │
│ Municipios                │
│ Municipios                │
│ Municipios                │
│ Distritos                 │
│ Distritos                 │
│ Distritos                 │
│ Distritos                 │
├───────────────────────────┤
│          10 rows          │
└───────────────────────────┘

In [103]:
con.sql("SELECT * FROM bronze_ine_renta")

┌────────────┬───────────────────────────────┬─────────┐
│   codigo   │             fecha             │  valor  │
│  varchar   │            varchar            │ double  │
├────────────┼───────────────────────────────┼─────────┤
│ 0200301010 │ 2023-01-01T00:00:00.000+01:00 │ 14275.0 │
│ 0200301010 │ 2022-01-01T00:00:00.000+01:00 │ 13632.0 │
│ 0200301010 │ 2021-01-01T00:00:00.000+01:00 │ 12728.0 │
│ 0200301010 │ 2020-01-01T00:00:00.000+01:00 │ 12384.0 │
│ 0200301010 │ 2019-01-01T00:00:00.000+01:00 │ 12092.0 │
│ 0200301010 │ 2018-01-01T00:00:00.000+01:00 │ 11276.0 │
│ 0200301010 │ 2017-01-01T00:00:00.000+01:00 │ 10554.0 │
│ 0200301010 │ 2016-01-01T00:00:00.000+01:00 │ 10195.0 │
│ 0200301010 │ 2015-01-01T00:00:00.000+01:00 │  9895.0 │
│ 0200302025 │ 2023-01-01T00:00:00.000+01:00 │ 16016.0 │
│     ·      │               ·               │     ·   │
│     ·      │               ·               │     ·   │
│     ·      │               ·               │     ·   │
│ 0207502001 │ 2015-01-01T00:00

In [64]:
con = duckdb.connect()

con.sql("INSTALL spatial; LOAD spatial;")
url = './data/mitma/zonificacion/distritos/zonificacion_distritos.shp'
con.sql(f"SELECT * FROM st_read('{url}') LIMIT 10")

┌──────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [5]:
con.sql("SELECT * FROM prueba")

┌──────────┬─────────┬──────────┬──────────┬───────────┬──────────────────┬───────────────────┬────────────────────────┬─────────────────────────┬────────────┬─────────┬─────────┬─────────┬─────────┬───────────┐
│  fecha   │ periodo │  origen  │ destino  │ distancia │ actividad_origen │ actividad_destino │ estudio_origen_posible │ estudio_destino_posible │ residencia │  renta  │  edad   │  sexo   │ viajes  │ viajes_km │
│ varchar  │ varchar │ varchar  │ varchar  │  varchar  │     varchar      │      varchar      │        varchar         │         varchar         │  varchar   │ varchar │ varchar │ varchar │ varchar │  varchar  │
├──────────┼─────────┼──────────┼──────────┼───────────┼──────────────────┼───────────────────┼────────────────────────┼─────────────────────────┼────────────┼─────────┼─────────┼─────────┼─────────┼───────────┤
│ 20230102 │ 00      │ 01001    │ 01001    │ 2-10      │ frecuente        │ frecuente         │ no                     │ no                      │ 01   

In [6]:
con.sql("SELECT count(*) FROM prueba")

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│     16988306 │
└──────────────┘

In [4]:
import duckdb
con = duckdb.connect()

In [37]:
con.sql("SELECT month(CAST(STRPTIME(source.fecha, '%Y%m%d') AS DATE)) as mes FROM prueba AS source WHERE mes =0")

┌────────┐
│  mes   │
│ int64  │
├────────┤
│ 0 rows │
└────────┘

In [ ]:
con.sql("""--sql
        SELECT *
        FROM INFORMATION_SCHEMA.COLUMNS
        WHERE TABLE_NAME = 'prueba'
        """)

In [1]:
from datetime import date, timedelta

start_date = date(2023, 1, 1) 
end_date = date(2023, 12, 31)    # perhaps date.today()

delta = end_date - start_date   # returns timedelta

for i in range(delta.days + 1):
    day = start_date + timedelta(days=i)
    print(day)

2023-01-01
2023-01-02
2023-01-03
2023-01-04
2023-01-05
2023-01-06
2023-01-07
2023-01-08
2023-01-09
2023-01-10
2023-01-11
2023-01-12
2023-01-13
2023-01-14
2023-01-15
2023-01-16
2023-01-17
2023-01-18
2023-01-19
2023-01-20
2023-01-21
2023-01-22
2023-01-23
2023-01-24
2023-01-25
2023-01-26
2023-01-27
2023-01-28
2023-01-29
2023-01-30
2023-01-31
2023-02-01
2023-02-02
2023-02-03
2023-02-04
2023-02-05
2023-02-06
2023-02-07
2023-02-08
2023-02-09
2023-02-10
2023-02-11
2023-02-12
2023-02-13
2023-02-14
2023-02-15
2023-02-16
2023-02-17
2023-02-18
2023-02-19
2023-02-20
2023-02-21
2023-02-22
2023-02-23
2023-02-24
2023-02-25
2023-02-26
2023-02-27
2023-02-28
2023-03-01
2023-03-02
2023-03-03
2023-03-04
2023-03-05
2023-03-06
2023-03-07
2023-03-08
2023-03-09
2023-03-10
2023-03-11
2023-03-12
2023-03-13
2023-03-14
2023-03-15
2023-03-16
2023-03-17
2023-03-18
2023-03-19
2023-03-20
2023-03-21
2023-03-22
2023-03-23
2023-03-24
2023-03-25
2023-03-26
2023-03-27
2023-03-28
2023-03-29
2023-03-30
2023-03-31
2023-04-01